# Simplest Colour Balance
## IPOL notebook demo (interactive version)

In [1]:
from pathlib import Path
from glob import glob

import io
from PIL import Image

import ipywidgets as widgets  # to select inputs
import imageio  # ipywidgets reads images as a byte array, imageio can then convert it to a more useful numpy array
from matplotlib import pyplot as plt  # Will be used to plot results

In [2]:
%matplotlib widget

In [3]:
import numpy as np  # used in the code just below

from simplest_color_balance import scb  # our code

In [4]:
import ipol

In [5]:
def histogram(img, ax):
    if img.ndim == 3:
        sz = img[:, :, 0].size
        hist_r, _ = np.histogram(img[:, :, 0].ravel(),256,[0,256])
        hist_g, _ = np.histogram(img[:, :, 1].ravel(),256,[0,256])
        hist_b, _ = np.histogram(img[:, :, 2].ravel(),256,[0,256])
        hist_r = hist_r / sz
        hist_g = hist_g / sz
        hist_b = hist_b / sz
        ax.plot(hist_r, color='red', lw=.8)
        ax.plot(hist_g, color='green', lw=.8)
        ax.plot(hist_b, color='blue', lw=.8)
    elif img.ndim == 1:
        hist, _ = np.histogram(img.ravel(), 256, [0, 256], lw=.8)
        ax.plot(hist)
    else:
        raise ValueError(img.ndim)

In [20]:
class Launcher(ipol.IPOLLauncher):
    def __init__(self):
        images = sorted(list(map(Path, glob('images/*'))))[:10]
        self.w_in_image = ipol.ImageInput(description="Input image",
                                           example_paths=images,
                                           example_labels=None,
                                           enable_upload=True,
                                           enable_crop=True)
        self.w_in_saturation = ipol.RangeTextSlider(0, 1,
                                                    default_values=(.015, .985),
                                                    value_type='float',
                                                    description_left='s0',
                                                    description_right='s1',
                                                    description='Saturation',
                                                    step=1e-3,
                                                    readout_format='.1%')
        in_widgets = [self.w_in_image, self.w_in_saturation]
        launch_params = {'img': self.w_in_image, 'saturation': self.w_in_saturation}
        super().__init__(launch_params, in_widgets)
        
    def launch(self, img, saturation):
        img = imageio.imread(img)
        s0, s1 = saturation
        out = self.run(img, s0, s1)
        self.display_results(img, out)
        # we could also have self.display_results return the histogram to register it
        self.register(input_image=img, s0=s0, s1=s1, result=out)
        
    def run(self, img, s0, s1):
        """
        The true code that runs the demo is here. The rest is only the I/O interface.
        For more complex code, it's possible to externalize functions in other cells, but they must be called by `run`
        (it is **not** possible to arbitrarily run cells).
        """
        out = scb(img, s0, 1-s1)
        return out
    
    def display_results(self, img, out):
        fig, ax = plt.subplots(2, 2, figsize=(10, 5))
        ax[0, 0].imshow(img)
        ax[0, 0].set_title('Input')
        ax[0, 0].axis('off')
        ax[0, 1].imshow(out)
        ax[0, 1].set_title('Output')
        ax[0, 1].axis('off')
        histogram(img, ax[1, 0])
        ax[1, 0].set_title('Input histogram')
        histogram((out*255).astype(np.uint8), ax[1, 1])
        ax[1, 1].set_title('Output histogram')
        plt.show()

In [21]:
launcher = Launcher()

In [22]:
launcher.display()